In [1]:
import re
import random
import pandas as pd
import nlpaug.augmenter.word as naw

from tqdm import tqdm
from nlpaug.util import Action
from koeda import EasyDataAugmentation, AEasierDataAugmentation
from googletrans import Translator

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import time

def timeit(func):
    """이 데코레이터는 함수의 실행 시간을 측정합니다."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 시작 시간 측정
        result = func(*args, **kwargs)
        end_time = time.time()    # 종료 시간 측정
        elapsed_time = end_time - start_time
        print(f"'{func.__name__}' 함수 실행 시간: {elapsed_time:.4f}초")
        return result
    return wrapper

In [3]:
df = pd.read_csv('../dataset/cleaned_train.csv')

In [4]:
idx = random.randint(0, 12000)
sample = df.iloc[idx]['dialogue']

In [5]:
@timeit
def augment_text_data_with_EDA(text: str, repetition=1):
    eda = EasyDataAugmentation(morpheme_analyzer="Mecab")
    
    pattern = r'#.*?#:?'
    
    lines = text.split('\n')
    augmented_texts = []
    
    for line in lines:
        matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, line)]
        cleaned_line = line
        for start, end, match in matches:
            cleaned_line = cleaned_line.replace(match, "")
        
        result = eda(cleaned_line, p=(0.1, 0.1, 0.1, 0.1), repetition=repetition)
        
        for start, end, match in matches:
            result = result[:start] + match + result[start:]
        
        augmented_texts.append(result.strip())

    return augmented_texts

In [6]:
# print(sample)
augment_text_data_with_EDA(sample)

'augment_text_data_with_EDA' 함수 실행 시간: 0.0038초


['#Person1#:야, 너. 좀 빨리 못 평원 해? 다들 너 때문에 기다리고 있어.',
 '#Person2#:어떻게 나한테만 그렇게 말? 이런 기계 조작하는 어려워. 오른손잡이용으로 디자인된 거야.',
 '#Person1#:너는 항상 이런 기계 때문에왼손불평해. 하지만 을 쓰 사람만 있는 것도는아닌데.',
 '#Person2#:정말? 나랑 같은 사람 다른알데 없는 줄 았어.',
 '#Person1#:사실 말하면 나 연령 도 왼손잡이야.',
 '#Person2#:너?',
 '#Person1#:그래. 미리 익숙해져야 돼. 그러면 빠르게 할 수 있.',
 '#Person2#:그게 너가 한 야거?',
 '#Person1#:그래. 사실 양손을 다 쓰는 게 좋아.',
 '#Person2#:알겠어, 일자 시도해볼게.']

In [7]:
@timeit
def augment_text_data_with_AEDA(text, repetition):
    """입력된 문장에 대해서 AEDA를 통해 데이터 증강"""
    aeda = AEasierDataAugmentation(morpheme_analyzer="Okt", punctuations=[".", ",", "!", "?", ";", ":"])

    pattern = r'#.*?#:?'
    lines = text.split('\n')
    augmented_texts = []
    
    for line in lines:
        matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, line)]
        
        cleaned_line = line
        for start, end, match in matches:
            cleaned_line = cleaned_line.replace(match, "")
    
        result = aeda(text, p=0.1, repetition=repetition)
    
        for start, end, match in matches:
            result = result[:start] + match + result[start:]
        
        augmented_texts.append(result.strip())

    return augmented_texts

In [8]:
augment_text_data_with_AEDA(sample, 1)

'augment_text_data_with_AEDA' 함수 실행 시간: 2.1973초


['#Person1#:#Person1#: 야, 너. 좀 빨리 못 해? 다들 너 때문에 기다리고 있어. #Person2#: 어떻게 나한테만 그렇게 말해? 이런 기계 조작하는 게 어려워. 오른손잡이용으로 . 디자인된 거야. #Person1#: 너는 항상 이런 , 기계 때문에 불평해. 하지만 왼손을 쓰는 사람만 있는 것도 아닌데. #Person2#: 정말? 나랑 같은 사람 다른 데 없는 줄 알았어 ! . #Person1#: 사실 말하면 나도 왼손잡이야. #Person2#: 너? #Person1#: 그래. 미리 익숙해져야 돼. 그러면 빠르게 할 수 있어 ? . #Person2#: 그게 너가 한 거야? #Person1#: 그래. ! 사실 양손을 다 쓰는 게 좋아. #Person2#: 알겠어, : 시도해볼게.',
 '#Person2#:#Person1#: 야, 너 ; . ? 좀 빨리 못 해? 다들 너 때문에 기다리고 ? 있어. #Person2#: 어떻게 나한테만 그렇게 말해 : ? ; 이런 기계 조작하는 게 어려워. 오른손잡이용으로 디자인된 거야. #Person1#: 너는 항상 이런 기계 때문에 불평해. 하지만 왼손을 쓰는 사람만 있는 것도 아닌데. #Person2#: 정말? 나랑 같은 사람 다른 데 없는 줄 알았어. #Person1#: 사실 말하면 나도 왼손잡이야. #Person2#: 너? #Person1#: 그래 ! . 미리 익숙해져야 돼. ! 그러면 빠르게 할 수 ; 있어. #Person2#: 그게 너 : 가 한 거야? ? #Person1#: 그래 ! . 사실 양손을 다 쓰는 게 좋아. . #Person2#: 알겠어, 시도해볼게.',
 '#Person1#:#Person1#: 야, 너. 좀 빨리 못 해? 다들 너 때문에 기다리고 있어. #Person2#: 어떻게 나한테만 그렇게 말해? 이런 기계 조작하는 게 어려워. 오른손잡이용으로 디자인된 거야. #Person1#: 너는 항상 이런 기계 때문에 불평해. 하지만 왼손을 쓰는 사람만 있는 것도 아닌데. #Person2#:

In [9]:
@timeit
def augment_text_data_with_BT(text, repetition):
    """입력된 문장에 대해서 BT를 통해 데이터 증강"""
    translator = Translator()
    result = []

    # 패턴 정의: #문자열#: 또는 #문자열# 추출
    pattern = r'#.*?#:?'
    matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, text)]
    
    cleaned_text = text
    for start, end, match in matches:
        cleaned_text = cleaned_text.replace(match, "")

    # 번역 실행 (한국어 > 영어 > 한국어)
    for i in range(repetition):
        translated = translator.translate(cleaned_text, src='ko', dest='en')
        re_translated = translator.translate(translated.text, src='en', dest='ko')
        augmented_text = re_translated.text
        
        for start, end, match in matches:
            augmented_text = augmented_text[:start] + match + augmented_text[start:]
        
        result.append(augmented_text)
    
    return result

In [10]:
augment_text_data_with_BT(sample, 1)

'augment_text_data_with_BT' 함수 실행 시간: 2.7491초


['#Person1#:이봐, 당신은 조금 빠르지 않습니까? 모두가 당신을 기다리고 있#Person2#:습니다.\n나에게 어떻게 그렇게 말합니까?이 기계를 조작하기가 어렵습니다. 오른손 히트를 위해 설계#Person1#:되었습니다.\n당신은 항상이 기계에 대해 불평하지만 왼손을 사용하는 사람은 아닙니다.\n정#Person2#:말? 나 같은 사람은 없다고 생각했습니다.\n사실, 나#Person1#:도 핸드를 남겼습니다.\n너?\n알겠#Person2#:습니다.#Person1#: 미리 익숙해 져야합니다. 그런 다음 빨리 할 수 \u200b\u200b있습니#Person2#:다.\n당신은 그렇게 했습#Person1#:니까?\n사실, 양손을 사용하는 것이 좋습니#Person2#:다.\n좋아요, 시도해 볼게요.']

In [11]:
def augment_text_data_with_NLPAug(text: str, repetition=1):
    aug = naw.ContextualWordEmbsAug(
        model_path='bert-base-multilingual-cased', action="substitute"
    )

    pattern = r'#.*?#:?'
    lines = text.split('\n')
    augmented_texts = []

    for line in lines:
        matches = [(m.start(), m.end(), m.group()) for m in re.finditer(pattern, line)]
        cleaned_line = line
        for start, end, match in matches:
            cleaned_line = cleaned_line.replace(match, "")
        
        result = []
        for _ in range(repetition):
            augmented_line = aug.augment(cleaned_line)
            for start, end, match in matches:
                augmented_line = augmented_line[:start] + match + augmented_line[start:]
            result.append(augmented_line.strip())

        augmented_texts.extend(result)

    return augmented_texts

In [12]:
# def augment_dataframe(df, repetition=1):
#     augmented_rows = []

#     for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Augmenting Data"):
#         original_dialogue = row['dialogue']
#         summary = row['summary']
#         topic = row['topic']

#         eda_augmented_texts = augment_text_data_with_EDA(original_dialogue, repetition)
#         aeda_augmented_texts = augment_text_data_with_AEDA(original_dialogue, repetition)
#         bt_augmented_texts = augment_text_data_with_BT(original_dialogue, repetition)

#         for text in eda_augmented_texts + bt_augmented_texts + aeda_augmented_texts:
#         # for text in eda_augmented_texts + bt_augmented_texts:
#             augmented_rows.append({
#                 'dialogue': text,
#                 'summary': summary,
#                 'topic': topic
#             })


#     augmented_df = pd.DataFrame(augmented_rows)
#     result_df = pd.concat([df, augmented_df], ignore_index=True)

#     return result_df

In [13]:
# augmented_df = augment_dataframe(df, repetition=1)

In [14]:
# augmented_df.to_csv('../dataset/augmented_train.csv', index=False)